OFAC SDN list - latest version (2023/12/12T09:42:00)

source url: https://ofac.treasury.gov/specially-designated-nationals-list-data-formats-data-schemas

Information on Data formats and definition of variables/ codebook: https://ofac.treasury.gov/media/29976/download?inline


*Data Cleaning Plan:*


1. Load differetnt data files from url
2. join together (control)
3. get overview of the entire file (column names, size of the df, frequencies, dtypes)
-> prob pivot "remark" column or extract information somehow
4. Transform Dtypes
5. Filter dataframe for missing data/ relevant columns
6. get descriptives & plot data

*Further Steps*

7. Renaming & Filtering
8. Creation of new Variables
9. Grouping


In [1]:
#import packages
import pandas as pd
import numpy as np
import os

In [24]:
#load the data
#first define the columns I want the data assigned to
columns = ["ent_num","SDN_Name","SDN_Type","Program","Title","Call_Sign","Vess_type","Tonnage","GRT","Vess_flag","Vess_owner","Remarks"]
df=pd.read_csv("https://www.treasury.gov/ofac/downloads/sdn.csv",names=columns,header=None)
df

,ent_num,SDN_Name,SDN_Type,Program,Title,Call_Sign,Vess_type,Tonnage,GRT,Vess_flag,Vess_owner,Remarks
0,36,AEROCARIBBEAN AIRLINES,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-
1,173,"ANGLO-CARIBBEAN CO., LTD.",-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-
2,306,BANCO NACIONAL DE CUBA,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,a.k.a. 'BNC'.
3,424,BOUTIQUE LA MAISON,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-
4,475,CASA DE CUBA,-0-,CUBA,-0-,-0-,-0-,-0-,-0-,-0-,-0-,-0-
...,...,...,...,...,...,...,...,...,...,...,...,...
13575,46880,HODROJ EXCHANGE S.A.R.L,-0-,SDGT,-0-,-0-,-0-,-0-,-0-,-0-,-0-,Secondary sanctions risk: section 1(b) of Exec...
13576,46884,"HU, Lianhe",individual,GLOMAG,-0-,-0-,-0-,-0-,-0-,-0-,-0-,"DOB 04 Oct 1967; POB Shaoyang, Hunan, China; n..."
13577,46888,"ARDESTANI, Mohammad Mahdi Khanpour",individual,IRAN-HR,-0-,-0-,-0-,-0-,-0-,-0-,-0-,DOB 21 Sep 1980; nationality Iran; Additional ...
13578,46889,"FARAHANI, Majid Dastjani",individual,IRAN-HR,-0-,-0-,-0-,-0-,-0-,-0-,-0-,DOB 26 Jul 1982; alt. DOB 27 Jul 1982; nationa...


In [27]:
df.ent_num.describe()

count     13580
unique    13580
top          36
freq          1
Name: ent_num, dtype: object

In [10]:
#now load the additional datasets, griup all of the by the ID and then join together
columns_add=["ent_num","add_num","address","city","state_province","postal_code","country""add_remarks"]
df_add = pd.read_csv("https://www.treasury.gov/ofac/downloads/add.csv",names=columns_add,header=None)
print(df_add)
columns_alt =["ent_num","alt_num","alt_type","alt_name","alt_remarks"]
df_alt =pd.read_csv("https://www.treasury.gov/ofac/downloads/alt.csv",names=columns_alt,header=None)
print(df_alt)

      ent_num  add_num                                            address  \
0          36     25.0                                               -0-    
1         173    129.0                           Ibex House, The Minories   
2         306    199.0                                   Zweierstrasse 35   
3         306    200.0                         Avenida de Concha Espina 8   
4         306    201.0  Dai-Ichi Bldg. 6th Floor, 10-2 Nihombashi, 2-c...   
...       ...      ...                                                ...   
18741   46888  69930.0                                               -0-    
18742   46888  69931.0                                               -0-    
18743   46889  69911.0                                               -0-    
18744   46889  69929.0                                               -0-    
18745             NaN                                                NaN   

                  city  state_province postal_code  countryadd_remarks  
0 

In [14]:
columns_comments=["ent_num","comments"]
df_comments =pd.read_csv("https://www.treasury.gov/ofac/downloads/sdn_comments.csv",names=columns_comments,header=None)
df_comments

,ent_num,comments
0,17018,: 24 Feb 2022; Effective Date (EO 14024 Direct...
1,27307,G TEAM'; a.k.a. 'RED DOT'; a.k.a. 'TEMP.HERMIT...
2,28263,hn'; Linked To: LAZARUS GROUP.
3,28264,hone Number 8613314257947; alt. Phone Number 8...
4,29703,s - ETH 0x8576acc5c05d6ce88f4e49bf65bdf0c62f91...
5,30518,ChmWqGtsaLrGbHfgdrV5Nkvahtjjuxr; alt. Digital ...
6,31212,Currency Address - BCH qq3vlashthktqpeppuv7tr...
7,33151,aa35MVkf7sDjuGUYAP5izQt7Qi; Digital Currency A...
8,33854,M7CGBPUJwXXSroWuZ6H5jiprdKCyf7V5M; alt. Digita...
9,36216,FhMdRq5UpPDpNyriYNVtkn; alt. Digital Currency ...


In [ ]:
#join all the dataframes together on the ent_num
df_sdn = pd.merge()

In [ ]:
#check if the joining was  successful -> sum up rows and do example query
df_sdn.query("ent_num==36")

In [ ]:
df_sdn.describe()

In [ ]:
df_sdn.columns.unique()

In [9]:
#get the percentages of missingsness
values_list = list()
cols_list = list() #creating empty lists

for col in df_sdn.columns:
    pct_missing= np.mean(df_sdn[col].isnull())*100
    cols_list.append(col)
    values_list.append(pct_missing)

pct_missing_df =pd.DataFrame()
pct_missing_df["col"] = cols_list
pct_missing_df["pct_missing"] = values_list
pct_missing_df
#shows a lot of missingness, prob need to use the advanced files or the additional files? 
#Or not use the compressed zip file?

,col,pct_missing
0,Publish_Date,99.992611
1,Record_Count,99.992611
2,uid,0.007389
3,lastName,0.007389
4,sdnType,0.007389
5,programList,100.000000
6,akaList,100.000000
7,addressList,100.000000
8,idList,100.000000
9,firstName,50.938377
